<a href="https://colab.research.google.com/github/mftorres/APP/blob/main/American_palm_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Leaving this here in case you need info about Colaboratory in Google

[link text](https://)<p><img alt="Colaboratory logo" height="45px" src="/img/colab_favicon.ico" align="left" hspace="10px" vspace="0px"></p>

<h1>What is Colaboratory?</h1>

Colaboratory, or "Colab" for short, allows you to write and execute Python in your browser, with 
- Zero configuration required
- Free access to GPUs
- Easy sharing

Whether you're a **student**, a **data scientist** or an **AI researcher**, Colab can make your work easier. Watch [Introduction to Colab](https://www.youtube.com/watch?v=inN8seMm7UI) to learn more, or just get started below!

----

----

# **American Palm Phylogeny data exploration/management**

Document authored by **Maria Fernanda Torres Jimenez**  
Date: 2021 May 05

____


The aim of this notebook is to show you how to retrieve data from the American Palm Phylogeny metadata, how to query, and how to update records.

Maintaining metadata associated to files requires you to do version control, properly dated, and to keep backups at all times. You should be prepared to accidentally change/delete metadata and you should always be able to go back to a stable version (but that depends on you and your data practices).


## 0. Importing packages

In [13]:
import pandas as pd
import re
import matplotlib as mpl
from matplotlib import pyplot as plt
import requests as rs
import datetime
import numpy as np

# !pip install fuzzywuzzy # install package to do partial text match
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

The database is stored in a google spreadsheet. It is good to have it in the cloud, google has version control and everyone can have access to the document without it staying with a single person.

Creating a function to download the data:

In [4]:
def get_data():
  return pd.read_csv('./palm_metadata_%s.txt'%(str(datetime.datetime.now()).split(' ')[0]), sep = '\t')

# try reading the file if exists, keep in mind the date
try:
  df = get_data()
except:
  url = 'https://raw.githubusercontent.com/mftorres/APP/main/data/Metadata_palm_gap_2021-05-05.txt' # shared link careful who you share this with
  print('Downloading')
  res = rs.get(url = url)
  with open('./palm_metadata_%s.txt'%(str(datetime.datetime.now()).split(' ')[0]), 'wb') as file: # Writing document in binary mode so python doesn't make changes
    file.write(res.content)
  df = get_data()

df # check the database

Downloading


,dataset,provider,pi_code,originalmetadata,filename,sense,filecode,samplecode,library_index,botanic_garden,voucher,taxgenus,taxspecies,ifmorphotype,ifpopulation,sent_to_Cano,newfilename,raw_reads,reads_trimed_paired_flag,reads_trimed_paired,reads_trimed_single,percentage_lost_tosingles,pcr_filtered,collection_year,Continent,country,long,lat,flag,notes
0,Cano,Angela Cano,AC,Sent Appendix_SamplingPhylogeny_CentralAmerica...,1_151124_000000000-AJE6N_P3252_1001_1.fastq.gz,R1,AJE6N_P3252_1001,AJE6N_1001,CGATGT,G,Cano_A._etal__ACS338,Acrocomia,Acrocomia_aculeata,NaN,NaN,sent,Acr_acu_AJE6N1001_AC_R1.fastq.gz,NaN,passed,NaN,NaN,NaN,NaN,NaN,americas,Panama,NaN,NaN,NaN,NaN
1,Cano,Angela Cano,AC,Sent Appendix_SamplingPhylogeny_CentralAmerica...,1_151124_000000000-AJE6N_P3252_1001_2.fastq.gz,R2,AJE6N_P3252_1001,AJE6N_1001,CGATGT,G,Cano_A._etal__ACS338,Acrocomia,Acrocomia_aculeata,NaN,NaN,sent,Acr_acu_AJE6N1001_AC_R2.fastq.gz,NaN,passed,NaN,NaN,NaN,NaN,NaN,americas,Panama,NaN,NaN,NaN,NaN
2,Cano,Angela Cano,AC,Sent Appendix_SamplingPhylogeny_CentralAmerica...,1_151130_000000000-AK5EU_P3252_1126_1.fastq.gz,R1,AK5EU_P3252_1126,AK5EU_1126,CACCGG,FTBG,_FTBG_20040120A,Acrocomia,Acrocomia_crispa,NaN,NaN,sent,Acr_cri_AK5EU1126_AC_R1.fastq.gz,NaN,failed,NaN,NaN,NaN,NaN,NaN,americas,NaN,NaN,NaN,NaN,NaN
3,Cano,Angela Cano,AC,Sent Appendix_SamplingPhylogeny_CentralAmerica...,1_151130_000000000-AK5EU_P3252_1126_2.fastq.gz,R2,AK5EU_P3252_1126,AK5EU_1126,CACCGG,FTBG,_FTBG_20040120A,Acrocomia,Acrocomia_crispa,NaN,NaN,sent,Acr_cri_AK5EU1126_AC_R2.fastq.gz,NaN,failed,NaN,NaN,NaN,NaN,NaN,americas,NaN,NaN,NaN,NaN,NaN
4,Cano,Angela Cano,AC,Sent Appendix_SamplingPhylogeny_CentralAmerica...,1_151130_000000000-AKG91_P3252_1182_1.fastq.gz,R1,AKG91_P3252_1182,AKG91_1182,CTAGCT,JBP,Lorenzi_&_Soares_6762,Acrocomia,Acrocomia_emensis,NaN,NaN,sent,Acr_eme_AKG911182_AC_R1.fastq.gz,NaN,passed,NaN,NaN,NaN,NaN,NaN,americas,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3044,EliottGardner,Eliott Gardner,EGR,none,Chamaedorea_vulgata_Standley67344_R2_001.fastq.gz,R2,Standley67344,NaN,NaN,NaN,NaN,Chamaedorea,Chamaedorea_vulgata,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3045,EliottGardner,Eliott Gardner,EGR,none,Chamaedorea_zamorae_Azofeifa484_R1_001.fastq.gz,R1,Azofeifa484,NaN,NaN,NaN,NaN,Chamaedorea,Chamaedorea_zamorae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3046,EliottGardner,Eliott Gardner,EGR,none,Chamaedorea_zamorae_Azofeifa484_R2_001.fastq.gz,R2,Azofeifa484,NaN,NaN,NaN,NaN,Chamaedorea,Chamaedorea_zamorae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3047,EliottGardner,Eliott Gardner,EGR,none,Chamaedorea_zamorae_Azofeifa484_S36_L001_R1_00...,R1,Azofeifa484_S36_L001,NaN,NaN,NaN,NaN,Chamaedorea,Chamaedorea_zamorae,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The first thing to check is the columns on the data and what they mean

In [5]:
list(df.columns)

['dataset',
 'provider',
 'pi_code',
 'originalmetadata',
 'filename',
 'sense',
 'filecode',
 'samplecode',
 'library_index',
 'botanic_garden',
 'voucher',
 'taxgenus',
 'taxspecies',
 'ifmorphotype',
 'ifpopulation',
 'sent_to_Cano',
 'newfilename',
 'raw_reads',
 'reads_trimed_paired_flag',
 'reads_trimed_paired',
 'reads_trimed_single',
 'percentage_lost_tosingles',
 'pcr_filtered',
 'collection_year',
 'Continent',
 'country',
 'long',
 'lat',
 'flag',
 'notes']

**dataset** bundle in which files are provided. e.g. CANO: sequences extracted by Angela Cano  
**provider** who provided the files or is responsible for sending the files  
**pi_code** two-letter code to shorten the *provider*  
**originalmetadata** name of original metadata provided by provider. That file should never be modified and only annotated  
**filename** original file name, usually as delivered by the sequencing facility  
**sense** whether file contains forward, reverse, interleaved, single reads    
**filecode** code provided by sequencing facility, often links to sequencing experiment, lane and plate information  
**samplecode** sample ID in the plate or as delivered to the sequencing facility
**library_index** sequencing index if provided or if in header of fastq. When double, each index is separated with a '+'  
**botanic_garden** if sample was obtained from a botanic garden rather than collected  
**voucher** voucher of the tissue or collecting code  
**taxgenus** taxonomic genus of the sample  
**taxspecies** taxonomic species with genus, separated by an underscore '_'  
**ifmorphotype** if the sample has a morphotype, variety or subspecies assigned  
**ifpopulation** if the sample belongs to a population-wide sampling effort, which population it was collected from  
**sent_to_Cano** files sent to Angela Cano who's analysing the data and leading the project  
**newfilename** new file name assigned to the sample. This is to annotate provider, dataset, species and voucher in the name. See Appendix for the code used to name files  
**raw_reads** number of raw reads before processing the data  -- filled as analysis progresses  
**reads_trimed_paired_flag** if trimmed -- filled as analysis progresses  
**reads_trimed_paired** number of reads after trimming -- filled as analysis progresses  
**reads_trimed_single** number of unpaired reads after trimming -- filled as analysis progresses  
**percentage_lost_tosingles** percentage of unpaired reads from the total of reads -- filled as analysis progresses  
**pcr_filtered** if PCR duplicate reads were filtered out using samtools or picard-- filled as analysis progresses  
**collection_year** year that sample was collected  
**Continent** continent. The project involves species from America but the metadata keeps track of other data from other continents  
**country** which country was the sample collected at if known from collection or voucher  
**long** longitude coordinates if available, should be standardised to decimals  
**lat**  latitude coordinates if available, should be standardised to decimals  
**flag** single word descriptors separated by spaces (never colons, semicolons or tabs) that describe the flaws of the file. e.g. misingtaxspecies  
**notes** elaborated notes if needed but avoid using commas, semicolons or tabs.  

The second interesting thing to do is filtering unique species for which we have data. The files here represent species that might or might not be good, but at this point of the analysis (which will be on a sequence by sequence basis) we can only know what we have sequenced.

Another thing is to filter out species that are not in America.

In [6]:
df['Continent'].unique()

array(['americas', 'out_americas', nan], dtype=object)

Some species don't have a continent specified. Let's see which ones:

In [8]:
# filtering pandas dataframe
df[df['Continent'].isna()]['taxspecies'].unique()

array(['Ceroxylon_alpinum', 'Ceroxylon_ceriferum',
       'Ceroxylon_echinulatum', 'Ceroxylon_parvifrons',
       'Ceroxylon_parvum', 'Ceroxylon_quindiuense',
       'Ceroxylon_ventricosum', 'Ceroxylon_vogelianum', 'Chamaedorea',
       'Cocos', nan, 'Phytelephas_aecuatorialis',
       'Phytelephas_macrocarpa', 'Phytelephas', 'Phytelephas_tumacana',
       'Ravenea_sambiranensis', 'Chamaedorea_nationsiana',
       'Chamaedorea_arenbergiana', 'Chamaedorea_binderi',
       'Chamaedorea_brachyclada', 'Chamaedorea_seifrizii',
       'Chamaedorea_fractiflexa', 'Chamaedorea_graminifolia',
       'Chamaedorea_hodelii', 'Chamaedorea_ibarrae',
       'Chamaedorea_incrustata', 'Chamaedorea_keelerorum',
       'Chamaedorea_lehmannii', 'Chamaedorea_liebmannii',
       'Chamaedorea_pachenoana', 'Chamaedorea_parvisecta',
       'Chamaedorea_pauciflora', 'Chamaedorea_piscifolia',
       'Chamaedorea_pumila', 'Chamaedorea_costaricana',
       'Chamaedorea_rigida', 'Chamaedorea_schideana',
       'Cham

In [9]:
# using a diferent syntax that can help with readability
df[df.Continent.isna()]['taxspecies'].unique()

array(['Ceroxylon_alpinum', 'Ceroxylon_ceriferum',
       'Ceroxylon_echinulatum', 'Ceroxylon_parvifrons',
       'Ceroxylon_parvum', 'Ceroxylon_quindiuense',
       'Ceroxylon_ventricosum', 'Ceroxylon_vogelianum', 'Chamaedorea',
       'Cocos', nan, 'Phytelephas_aecuatorialis',
       'Phytelephas_macrocarpa', 'Phytelephas', 'Phytelephas_tumacana',
       'Ravenea_sambiranensis', 'Chamaedorea_nationsiana',
       'Chamaedorea_arenbergiana', 'Chamaedorea_binderi',
       'Chamaedorea_brachyclada', 'Chamaedorea_seifrizii',
       'Chamaedorea_fractiflexa', 'Chamaedorea_graminifolia',
       'Chamaedorea_hodelii', 'Chamaedorea_ibarrae',
       'Chamaedorea_incrustata', 'Chamaedorea_keelerorum',
       'Chamaedorea_lehmannii', 'Chamaedorea_liebmannii',
       'Chamaedorea_pachenoana', 'Chamaedorea_parvisecta',
       'Chamaedorea_pauciflora', 'Chamaedorea_piscifolia',
       'Chamaedorea_pumila', 'Chamaedorea_costaricana',
       'Chamaedorea_rigida', 'Chamaedorea_schideana',
       'Cham

In [10]:
df[df.Continent.isna()]['taxgenus'].unique()

array(['Ceroxylon', 'Chamaedorea', 'Cocos', nan, 'Phytelephas', 'Ravenea'],
      dtype=object)

Most species are from the genera *Ceroxylon*, *Chamaedorea*, *Cocos*, *Phytelephas*, and *Ravenea*. I can already remember the dataset they come from and I can tell they don't have a continent assigned because I haven't updated the dataframe (as is not my main priority)

we can totally fix that today though. Let's iterate through rows in the dataset to assign the continent information. We also know all but *Cocos* (which is cosmopolita) are from America.

In [31]:
# itertuples iterates through tuples (pairs) of index,row and different columns can be acceced using the column label. Similar to the iterrows() method but faster.
# itertuples() only allows for the "dot" syntax for filtering pandas columns.

for row in df[df['Continent'].isna()].itertuples():
  print(row.Continent,'empty') # test
  df.loc[row.Index,'Continent']
  # else:
    # print(row.Continent)

nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty
nan empty


You can import your own data into Colab notebooks from your Google Drive account, including from spreadsheets, as well as from Github and many other sources. To learn more about importing data, and how Colab can be used for data science, see the links below under [Working with Data](#working-with-data).